# Trabalho final para a matéria: "Anotação de Textos e Processamento de Linguagem Natural (CIC0087)" 😊

## Tentativa de reconhecimento de todas as Entidades Nomeadas providas no corpo de contratos.

## Sim, me dei conta de que o pedido da tarefa originalmente era a classificação de algumas entidades definidas, mas tentei com meus conhecimentos tentar classificar tudo para ver o que aconteceria, como um treinamento para a execução da classificação final.

### 👨‍💻 Aluno: Guilherme Braga (17/0162290)

#### 🐱 [Repositório no Github](https://github.com/gui1080/IA_CIC0087_Reconhecimento_de_Entidades_Nomeadas)

### Pegando os dados originais 🧐

In [1]:
from google.colab import files

# importar o arquivo de disco local, com o nome original "contratos.csv"
uploaded = files.upload()

Saving contratos.csv to contratos.csv


### Importando, separando e limpando os dados 🚮


In [2]:
import pandas as pd
import numpy as np
import io
from sklearn.utils import shuffle

# Adicionando em dataframe
df = pd.read_csv(io.BytesIO(uploaded['contratos.csv']))
df = df.replace([np.inf, -np.inf], np.nan)
df = df.reset_index()

df

,index,id_dodf,tipo_rel,id_rel,anotador_rel,timestamp_rel,tipo_ent,id_ent,anotador_ent,timestamp_ent,offset,length,texto
0,0,22_20.12.2019.1,REL_ADITAMENTO_CONTRATO,R5,matheus_santos,2022-04-02T22:45:52Z,numero_termo_aditivo,275,matheus_santos,2022-03-04T15:34:41Z,372969,20,QUARTO TERMO ADITIVO
1,1,22_20.12.2019.1,REL_ADITAMENTO_CONTRATO,R5,matheus_santos,2022-04-02T22:45:52Z,numero_contrato,276,matheus_santos,2022-03-04T15:34:41Z,373005,8,002/2018
2,2,22_20.12.2019.1,REL_ADITAMENTO_CONTRATO,R5,matheus_santos,2022-04-02T22:45:52Z,orgao_contratante,277,matheus_santos,2022-03-04T15:34:41Z,373098,8,DF/RA II
3,3,22_20.12.2019.1,REL_ADITAMENTO_CONTRATO,R5,matheus_santos,2022-04-02T22:45:52Z,objeto_aditamento_contratual,278,matheus_santos,2022-03-04T15:34:41Z,373238,481,O presente Termo Aditivo objetiva a alteracao ...
4,4,22_20.12.2019.1,REL_ADITAMENTO_CONTRATO,R5,matheus_santos,2022-04-02T22:45:52Z,processo_gdf,434,matheus_santos,2022-04-02T22:45:46Z,373072,16,0131-000016/2018
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6691,6691,8_13.8.2019,REL_EXTRATO_CONTRATO,R5,alexandre_santana,2022-03-08T12:03:42Z,vigencia_contrato,6,alexandre_santana,2022-02-22T02:34:10Z,478144,17,04 (quatro) meses
6692,6692,8_13.8.2019,REL_EXTRATO_CONTRATO,R5,alexandre_santana,2022-03-08T12:03:42Z,valor_contrato,5,alexandre_santana,2022-02-22T02:34:10Z,478173,9,"76.500,00"
6693,6693,8_13.8.2019,REL_EXTRATO_CONTRATO,R5,alexandre_santana,2022-03-08T12:03:42Z,data_assinatura_contrato,149,alexandre_santana,2022-03-08T12:03:37Z,478086,10,07/08/2019
6694,6694,8_13.8.2019,REL_EXTRATO_CONTRATO,R5,alexandre_santana,2022-03-08T12:03:42Z,objeto_contrato,150,alexandre_santana,2022-03-08T12:03:37Z,478106,26,aquisicao de fita eletrica


### Nem sempre é necessária a limpeza total dos dados! 😕

#### ["Aqui entidades numéricas, capitalização e pontuação podem ser importantes. Por isso, pensando em uma tarefa de classificação, faz sentido não adotar alguns dos passos de pré-processamento que normalmente são feitos."](https://colab.research.google.com/drive/1zjcTnNNLlbM3BH3zEAAnQSJjlUmTP8K2?usp=sharing#scrollTo=XP3U0XWKsmSf://)

In [3]:
# 80% vai ser treino, 20% vai ser teste
# 0.8 * 6695 = 5356
# até o índice 5356 vai ser treino, depois disso é teste

# é o dado
x = df.filter(['texto'])

x_train_raw = x.iloc[:5356] 
x_test_raw = x.iloc[5356:]  

# é o resultado do dado
y = df.filter(['id_ent'])

y_train_raw = y.iloc[:5356]
y_test_raw = y.iloc[5356:]


# preciso transformar o texto num numero

frases = df.filter(['texto'])


valores_distintos = y_train_raw['id_ent'].nunique() + 1
valores_distintos_nao_nulos = y_train_raw['id_ent'].count()

print(valores_distintos)              # entidades possíveis
print(valores_distintos_nao_nulos)    # repetições


624
5356


### Conversão para "one-hot codes" 🔥

In [4]:
from tensorflow import keras

# conversão para "one-hot codes"

# to_categorical -> vai de 0 até "valores_distintos - 1" como uma possibilidade
# esses dados começam em 1, vou subtrair 1 de tudo, para 0 virar uma categoria

y_train_raw = y_train_raw - 1
y_test_raw = y_test_raw - 1

y_train = keras.utils.to_categorical(y_train_raw, valores_distintos+1)
y_test = keras.utils.to_categorical(y_test_raw, valores_distintos+1)

print(y_train[0])


[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.

### Testarei inicialmente a stemnminzação, retirada de acentos (para garantir) e a retirada de stopwords da parte do texto. Na retirada de stopwords, não contarei com a retirada de "\n".

In [5]:
!pip install unidecode
import unidecode

     |████████████████████████████████| 235 kB 10.5 MB/s 


In [6]:
# retirada de acentos

frases['texto'] = frases['texto'].apply(lambda x: ' '.join([unidecode.unidecode(termo) for termo in x.split()]))
x_train_raw['texto'] = x_train_raw['texto'].apply(lambda x: ' '.join([unidecode.unidecode(termo) for termo in x.split()]))
x_test_raw['texto'] = x_test_raw['texto'].apply(lambda x: ' '.join([unidecode.unidecode(termo) for termo in x.split()]))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [7]:
import nltk
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('portuguese')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [8]:
# retirada de stopwords

frases['texto'] = frases['texto'].apply(lambda x: ' '.join([termo for termo in x.split() if termo not in stopwords]))
x_train_raw['texto'] = x_train_raw['texto'].apply(lambda x: ' '.join([termo for termo in x.split() if termo not in stopwords]))
x_test_raw['texto'] = x_test_raw['texto'].apply(lambda x: ' '.join([termo for termo in x.split() if termo not in stopwords]))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [9]:
nltk.download('rslp')
stemmer = nltk.stem.RSLPStemmer()

[nltk_data] Downloading package rslp to /root/nltk_data...
[nltk_data]   Unzipping stemmers/rslp.zip.


In [10]:
# steminização

frases['texto'] = frases['texto'].apply(lambda x: ' '.join(stemmer.stem(termo) for termo in x.split()))
x_train_raw['texto'] = x_train_raw['texto'].apply(lambda x: ' '.join(stemmer.stem(termo) for termo in x.split()))
x_test_raw['texto'] = x_test_raw['texto'].apply(lambda x: ' '.join(stemmer.stem(termo) for termo in x.split()))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [11]:
words_raw = set()

for i in range(5356):
    # printing a third element of column
    nossa_string = frases['texto'][i]
    for termo in nossa_string.split():
      words_raw.add(termo)

words_raw = list(words_raw)
num_words = len(words_raw)

def remove_dups(lista):
    """Remove os itens duplicados mantendo a ordem original."""
    return list(dict.fromkeys(lista))

words = remove_dups(words_raw)
words.sort()
num_words = len(words)

# lista de palavras possíveis
num_words

11286

# Mapeando texto em valores numéricos para alimentar a rede neural 🦜

Minha idéia: **Polynomial Rolling Hash**, como explicado neste [link](https://www.geeksforgeeks.org/string-hashing-using-polynomial-rolling-hash-function/).

In [12]:
# Limpeza pronta!

# eu tenho uma lista de palavras, "words"
# agora quero mapear cada palavra em x para um valor inteiro nessa lista


class Hash:
    def __init__(self, s: str):
        self.hash_value = 0
        self.p, self.m = 29791, 10**9 + 7
        self.length = len(s)
        hash_so_far = 0
        p_pow = 1
        for i in range(self.length):
            hash_so_far = (hash_so_far + (1 + ord(s[i]) - ord('a')) * p_pow) % self.m
            p_pow = (p_pow * self.p) % self.m
        self.hash_value = hash_so_far
     
    def __eq__(self, other):
        return self.hash_value == other.hash_value
 


def retorna_valor(frase):

  j = 0
  contador = 0

  acumulado = 0

  for termo in frase.split():
    
    j = Hash(termo)
    acumulado = acumulado + j.hash_value
    contador = contador + 1

  if(contador != 0):
    resultado = acumulado/contador
  else:
    resultado = acumulado

  return resultado

x_train = x_train_raw
x_test = x_test_raw

x_train['texto'] = x_train_raw['texto'].apply(retorna_valor)
x_test['texto'] = x_test_raw['texto'].apply(retorna_valor)

# vamos realizar a visualização do que vai servir de treino
x_train

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,texto
0,3.566527e+08
1,6.612621e+08
2,3.541638e+08
3,5.749177e+08
4,7.911207e+08
...,...
5351,8.515923e+08
5352,1.779231e+08
5353,5.163980e+08
5354,9.074186e+08


In [13]:
# deve-se mapear o menor e o maior valor para um range entre (x e y)

from scipy.interpolate import interp1d

max_index_train = x_train['texto'].max()
max_index_test = x_test['texto'].max()

if(max_index_train > max_index_test):
  max = max_index_train
else:
  max = max_index_test

min_index_train = x_train['texto'].max()
min_index_test = x_test['texto'].max()

if(min_index_train < min_index_test):
  min = min_index_train
else:
  min = min_index_test

print("Maior valor retornado -> " + str(max))
print("Menor valor retornado -> " + str(min))

def new_map(s):
  a = [min, max]
  b = [0, 4096]
  (a1, a2), (b1, b2) = a, b
  return  b1 + ((s - a1) * (b2 - b1) / (a2 - a1))


#x_train['texto'] = x_train['texto'].apply(new_map)
#x_test['texto'] = x_test['texto'].apply(new_map)


Maior valor retornado -> 999999965.0
Menor valor retornado -> 999999964.0


In [14]:
# vamos realizar a visualização do que vai servir de teste

x_test

,texto
5356,4.559366e+08
5357,4.411848e+08
5358,6.520701e+08
5359,8.489808e+08
5360,1.914764e+08
...,...
6691,8.438076e+08
6692,6.336678e+08
6693,4.264069e+08
6694,8.278445e+08


In [15]:
# vamos realizar a visualização do que vai servir de treino

x_train

,texto
0,3.566527e+08
1,6.612621e+08
2,3.541638e+08
3,5.749177e+08
4,7.911207e+08
...,...
5351,8.515923e+08
5352,1.779231e+08
5353,5.163980e+08
5354,9.074186e+08


In [16]:
num_words

11286

In [17]:
print(x_train.shape)
print(x_test.shape)

(5356, 1)
(1340, 1)


In [18]:
# Definindo shape correto

import numpy 

print(y_train.shape)
y_train = numpy.delete(y_train, (0), axis=1)
y_test = numpy.delete(y_test, (0), axis=1)
print(y_train.shape)

(5356, 625)
(5356, 624)


# RNN: Recurrent Neural Network ♟

### Nesse tipo de implementação, sequência e ordem importam!

#### Qual deveria ser a proporção escolhida perfeita para essa rede neural é uma pergunta complexa.


In [19]:
x_train = np.expand_dims(x_train, axis=-1)
y_train = np.expand_dims(y_train, axis=1)
print(x_train.shape)
print(y_train.shape)

(5356, 1, 1)
(5356, 1, 624)


In [20]:
print(x_test.shape)
print(y_test.shape)


print(x_test.shape)
print(y_test.shape)

(1340, 1)
(1340, 624)
(1340, 1)
(1340, 624)


# Criando a rede neural 🧠 

## Dúvida: como deve ser dada a arquitetura da minha rede? 🤔

A rede neural terá uma camada de input com tamanho proporcional à entrada, e uma camada de saída com tamanho proporcional à classificação que será feita. 

Acertar a quantidade de "hidden layers" e quantos nós ela deve ter também é uma tarefa complexa. Dados linearmente separáveis não precisam de uma rede neural com "hidden layer", o que certamente não é este caso. Errar "para mais" não é uma solução otimizada assim como "errar para menos" também não é. 

[Adotarei este cálculo](https://stackoverflow.com/questions/10565868/multi-layer-perceptron-mlp-architecture-criteria-for-choosing-number-of-hidde) para ter um chute decente de quantos nós devo adotar:

(num inputs + num outputs) x 2/3

(5356 entradas + 624 classificações) x 2/3 = 3,986

Arredondando ordens de grandes após alguns testes, farei uma rede com a seguinte estrutura de "hidden layers":

4992 x 4992 x 2496 x 1248 x 1248 

In [21]:
#nx_train = x_train.to_numpy()
#ny_train = y_train.to_numpy()
#nx_test = x_test.to_numpy()
#ny_test = y_test.to_numpy()

#nx_train.reshape(1, 5356, 1)
#ny_train.reshape(1, 5356, 625)
#nx_test.reshape(1, 5356, 1)
#zny_test.reshape(1, 5356, 625)

# 28 sequencias de 28 elementos

#x_train = np.expand_dims(x_train, axis=-1)
#y_train = np.expand_dims(y_train, axis=-1)

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM

# Input(shape = (T,D)) for RNN
# T = T is length of time sequence
# D = D is the number of features

# x_train.shape[1:] => (1,)

# input_shape=(x_train.shape[1:])
# input_shape=(5356, 1)

model = Sequential()

model.add(LSTM(4992, input_shape=(x_train.shape[1:]), activation='relu', return_sequences=True))
model.add(Dropout(0.2))

model.add(LSTM(4992, activation='relu', return_sequences=True))
model.add(Dropout(0.1))

model.add(Dense(2496, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(1248, activation='relu'))
model.add(Dropout(0.1))

model.add(Dense(1248, activation='relu'))
model.add(Dropout(0.1))

model.add(Dense(valores_distintos, activation='softmax'))




# Treinamento da RNN 🏋🏻‍♀️ 



In [ ]:
print(x_train.shape)
print(y_train.shape)

x_test = np.expand_dims(x_test, axis=-1)
y_test = np.expand_dims(y_test, axis=1)

opt = tf.keras.optimizers.Adam(learning_rate=0.1, decay=1e-3)

model.compile(
   loss='categorical_crossentropy',
   optimizer=opt,
   metrics=['accuracy'],
)

model.fit(x_train,
          y_train,
          epochs=10,
          validation_data=(x_test, y_test))

(5356, 1, 1)
(5356, 1, 624)
Epoch 1/10
168/168 [==============================] - 325s 2s/step - loss: 2329280512000.0000 - accuracy: 9.3353e-04 - val_loss: 6.0977 - val_accuracy: 0.0045
Epoch 2/10
168/168 [==============================] - 324s 2s/step - loss: 7.6475 - accuracy: 0.0015 - val_loss: 6.0736 - val_accuracy: 0.0030
Epoch 3/10
168/168 [==============================] - 328s 2s/step - loss: 7.2620 - accuracy: 0.0017 - val_loss: 6.0719 - val_accuracy: 0.0037
Epoch 4/10
168/168 [==============================] - 328s 2s/step - loss: 7.2352 - accuracy: 0.0022 - val_loss: 6.0623 - val_accuracy: 0.0037
Epoch 5/10
168/168 [==============================] - 351s 2s/step - loss: 675965960192.0000 - accuracy: 0.0022 - val_loss: 6.0693 - val_accuracy: 0.0030
Epoch 6/10
168/168 [==============================] - 331s 2s/step - loss: 7994565120.0000 - accuracy: 0.0024 - val_loss: 6.0615 - val_accuracy: 0.0037
Epoch 7/10
168/168 [==============================] - 334s 2s/step - loss: 6.3

# Avaliação do modelo 🤙🏻

In [ ]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

# Resultados 😳

Para 624 classificações, um chute aleatório nos dá a chance de acerto de 1/625.

No mínimo, os testes mostram que o desempenho da RNN é tipo um chute.

*A expectativa é que esta estrutura para uma quantidade menor de classificações tenha um desempenho superior.*